# Topic modelling of news headlines for prediction of news category with 6 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 6 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title_description_text' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_description_text_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<67513x29998 sparse matrix of type '<class 'numpy.int64'>'
	with 1026097 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=6, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

29998

In [14]:
cv.get_feature_names_out()

array(['000', '01', '034', ..., 'đoković', 'войны', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

6

In [16]:
lda.components_

array([[ 4.65770126,  0.16690303, 32.05245157, ...,  0.16673374,
         0.16667028,  5.16665164],
       [26.65850161,  0.16745963,  1.11436323, ...,  0.16716151,
         3.16664744,  0.16667027],
       [ 2.18952282,  1.1662503 , 20.78679717, ...,  3.16586354,
         0.16667026,  0.16666929],
       [ 6.02067306,  3.16577162,  1.25047601, ...,  0.1669073 ,
         0.16667192,  0.16667051],
       [ 6.44166401,  0.1666673 , 10.80989339, ...,  0.16666699,
         0.16667044,  0.16666943],
       [15.03193724,  0.16694813, 23.98601864, ...,  0.16666692,
         0.16666967,  0.16666886]])

In [17]:
len(lda.components_[0])

29998

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 1589, 12143, 17489, ..., 21156, 14353, 25501], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

präsident
selenskyj
moskau
kiew
ukrainisch
putin
russisch
russland
krieg
ukraine


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['deutschland', 'soldat', 'liveblog', 'angriff', 'entwicklung', 'präsident', 'selenskyj', 'moskau', 'kiew', 'ukrainisch', 'putin', 'russisch', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['stehen', 'spiel', 'erster', 'sieg', 'dfb', 'partei', 'fc', 'bundesliga', 'frau', 'gewinnen', 'deutsch', 'fußball', 'bayern', 'wm', 'afd']


Die TOP-15 Wörter für das Thema #2
['berliner', 'stehen', 'deutschland', 'groß', 'kanzler', 'wahl', 'gut', 'cdu', 'geben', 'spd', 'deutsch', 'online', 'scholz', 'focus', 'berlin']


Die TOP-15 Wörter für das Thema #3
['deutschland', 'scholz', 'geben', 'nato', 'biden', 'gipfel', 'usa', 'baerbock', 'türkei', 'us', 'präsident', 'land', 'china', 'corona', 'eu']


Die TOP-15 Wörter für das Thema #4
['taiwan', 'geben', 'iran', 'tote', 'regierung', 'sterben', 'gericht', 'ex', 'polizei', 'usa', 'protest', 'trump', 'präsident', 'mensch', 'us']


Die TOP-15 Wörter für das Thema #5
['million', 'mensch', 'groß', 'gu

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 6)

In [24]:
topic_results[0].round(5)

array([0.00732, 0.00728, 0.20736, 0.00744, 0.00727, 0.76333])

In [25]:
topic_results[0].argmax()

5

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,5
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,2
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,3


## Visualize model

In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5     -53.367214  16.534853       1        1  21.669788
2      78.260132  -1.989677       2        1  18.159449
0      19.258986  55.283031       3        1  17.825487
4     -38.987747 -64.517448       4        1  17.192973
3      42.260666 -75.869255       5        1  12.578741
1       9.482670 -14.049941       6        1  12.573563, topic_info=           Term          Freq         Total Category  logprob  loglift
25501   ukraine  11711.000000  11711.000000  Default  30.0000  30.0000
14353     krieg   5894.000000   5894.000000  Default  29.0000  29.0000
21156  russland   5946.000000   5946.000000  Default  28.0000  28.0000
21150  russisch   5607.000000   5607.000000  Default  27.0000  27.0000
19899     putin   3914.000000   3914.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
10573      groß    511.820399   3643.678520   Topic6  -5.6572   0.1108
16019      mann    415.605677   1561.143265   Topic6  -5.8655   0.7501
9307      geben    486.115554   5109.663176   Topic6  -5.7088  -0.2789
3938    bleiben    403.459654   2183.819269   Topic6  -5.8951   0.3848
10761       gut    379.129942   2737.676241   Topic6  -5.9573   0.0966

[430 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
627        1  0.001382             afd
627        6  0.998037             afd
641        5  0.993212      afghanisch
643        5  0.997579     afghanistan
706        1  0.997089        aiwanger
...      ...       ...             ...
29064      4  0.103857          zeigen
29064      5  0.083706          zeigen
29064      6  0.129046          zeigen
29934      3  0.005704  überschwemmung
29934      4  0.992561  überschwemmung

[891 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 1, 5, 4, 2])

## Save train set with topics and dump countvectorizer and model

In [29]:
#Save result to csv
train.to_csv('train_lda_combined_6.csv')

In [30]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv_combined.jl')

['models/cv_combined.jl']

In [31]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_6_topics_combined.jl')

['models/lda_model_6_topics_combined.jl']